## Deliverable 2. Create a Customer Travel Destinations Map.

In [12]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Yumen,CN,40.2833,97.2000,19.38,38,0,15.26
1,1,Basco,PH,20.4487,121.9702,75.81,68,63,23.49
2,2,Calamar,CO,10.2511,-74.9146,80.55,65,94,11.61
3,3,Barrow,US,71.2906,-156.7887,-12.98,77,1,13.80
4,4,Port Pirie,AU,-33.1833,138.0167,66.29,68,10,11.07


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.tail(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
674,674,Vao,NC,-22.6667,167.4833,75.67,93,100,14.32
675,675,Ambulu,ID,-8.3450,113.6058,78.82,87,88,1.68
680,680,Coracao De Jesus,BR,-16.6853,-44.3650,75.92,95,100,6.08
681,681,Sinnamary,GF,5.3833,-52.9500,77.09,90,100,10.58
682,682,Pundaguitan,PH,6.3711,126.1689,81.95,77,99,6.06
685,685,Maracaibo,VE,10.6317,-71.6406,78.80,72,97,10.51
689,689,Ho Chi Minh City,VN,10.7500,106.6667,77.02,73,40,4.61
692,692,Harper,LR,4.3750,-7.7169,79.90,83,39,7.61
694,694,Luis Correia,BR,-2.8792,-41.6669,86.00,74,20,18.41
696,696,Chaiyaphum,TH,15.7500,102.0000,75.92,61,89,14.94


In [13]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID       197
City          197
Country       196
Lat           197
Lng           197
Max Temp      197
Humidity      197
Cloudiness    197
Wind Speed    197
dtype: int64

In [17]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df=preferred_cities_df.dropna()
clean_preferred_cities_df.count()

City_ID       196
City          196
Country       196
Lat           196
Lng           196
Max Temp      196
Humidity      196
Cloudiness    196
Wind Speed    196
dtype: int64

In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Basco,PH,75.81,20.4487,121.9702,
2,Calamar,CO,80.55,10.2511,-74.9146,
9,Rikitea,PF,77.31,-23.1203,-134.9692,
14,Itarema,BR,80.85,-2.9248,-39.9167,
20,Avarua,CK,75.25,-21.2078,-159.7750,
22,Hilo,US,78.62,19.7297,-155.0900,
23,Kudahuvadhoo,MV,80.65,2.6708,72.8944,
28,Puerto Ayora,EC,75.16,-0.7393,-90.3518,
30,Butaritari,KI,82.53,3.0707,172.7902,
33,Vaini,TO,82.56,-21.2000,-175.2000,


In [29]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")
        

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [39]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df[hotel_df["Hotel Name"].str.len()>0]
clean_hotel_df.count()

City          180
Country       180
Max Temp      180
Lat           180
Lng           180
Hotel Name    180
dtype: int64

In [40]:
# 8a. Create the output File (CSV)
output_data_file="Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file,index_label="City_ID")

In [41]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [42]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))